## Module submission header
### Submission preparation instructions
_Completion of this header is mandatory, subject to a 2-point deduction to the assignment._ Only add plain text in the designated areas, i.e., replacing the relevant 'NA's. You must fill out all group member Names and Drexel email addresses in the below markdown list, under header __Module submission group__. It is required to fill out descriptive notes pertaining to any tutoring support received in the completion of this submission under the __Additional submission comments__ section at the bottom of the header. If no tutoring support was received, leave NA in place. You may as well list other optional comments pertaining to the submission at bottom. _Any distruption of this header's formatting will make your group liable to the 2-point deduction._

### Module submission group
- Group member 1
    - Name: Marissa Lynch
    - Email: ml3758@drexel.edu
- Group member 2
    - Name: Uditi Shah
    - Email: us54@drexel.edu
- Group member 3
    - Name: Yifan Wang
    - Email: yw827@drexel.edu

### Additional submission comments
- Tutoring support received: NA
- Other (other): NA

# Assignment group 1: Textual feature extraction and numerical comparison

## Module C _(35 points)_ Similarity of word usage across a document

Here we'll be building up some code to discover how different terms are utilized similarly across a document. For this, our first task will be to create a word frequency counting function.

__C1.__ _(12 points)_ Define a function called `count_words(paragraph, pos = True, lemma = True)` that `return`s a `Counter()` called `frequency`. In `frequency`, each key will consist of a `heading = (text, tag)`, where `text` contains the `word.text` attribute from `spacy` if `lemma = False`, and `word.lemma_` attribute if `True`. Similarly, `tag` should be left empty as `""` if `pos = False` and otherwise contain `word.pos_`. The `Counter()` should simply contain the number of times each `heading` is observed in the `paragraph`.

In [33]:
# C1:Function(12/12)

from collections import Counter
import spacy, json, re
import numpy as np

nlp = spacy.load("en_core_web_sm")

def count_words(paragraph, pos = True, lemma = True):

    #---Your code starts here
      doc = nlp(paragraph)
      frequency = Counter()

      for word in doc:
        text = word.lemma_ if lemma else word.text

        tag = word.pos_ if pos else ""

        heading = (text, tag)

        frequency[heading] += 1
    #---Your code ends here

      return frequency

Let's make sure your function works by testing it on a short sentence.

In [ ]:
# C1:SanityCheck

count_words("The quick brown fox jumps over the lazy dog.")

Counter({('the', 'DET'): 2,
         ('quick', 'ADJ'): 1,
         ('brown', 'ADJ'): 1,
         ('fox', 'NOUN'): 1,
         ('jump', 'VERB'): 1,
         ('over', 'ADP'): 1,
         ('lazy', 'ADJ'): 1,
         ('dog', 'NOUN'): 1,
         ('.', 'PUNCT'): 1})

__C2.__ _(8 pts)_ Next, define a function called `book_TDM(book_id, pos = True, lemma = True)` and copy into it the TDM-producing code from __Section 2.1.5.1__ of the lecture notes, now `return`-ing `TDM` and `all_words`. Once copied, modify this function to call `count_words` appropriately, now passing through the user of `book_TDM`'s specified `lemma` and `pos` arguments.

In [36]:
# C2:Function(8/8)

import numpy as np
from collections import Counter
import re
import os
import spacy

def book_TDM(book_id, pos = True, lemma = True):

    #---Your code starts here---
    file_path = os.path.join('sample_data', f'{book_id}.txt')

    with open(file_path, 'r', encoding='utf-8') as file:
        book_content = file.read()

    paragraphs = book_content.split('\n\n')

    all_words = []

    term_document_counts = []

    for paragraph in paragraphs:
        frequency = count_words(paragraph, pos, lemma)
        all_words.extend(frequency.keys())
        term_document_counts.append(frequency)

    all_words = list(set(all_words))

    TDM = np.zeros((len(all_words), len(paragraphs)))

    for j, frequency in enumerate(term_document_counts):
        for i, word in enumerate(all_words):
            TDM[i, j] = frequency[word]
    #---Your code ends here---

    return(TDM, all_words)


To test your code's function, let's process `book_id = 84` with both of `pos = True` and `lemma = True` and print out the `TDM`'s `.shape` attribute and the first ten terms in `all_words`.

In [37]:
# C2:SanityCheck

TDM, terms = book_TDM("84", pos = True, lemma = True)
terms[:10]

[('retreat', 'NOUN'),
 ('triumphant', 'ADJ'),
 ('dizzy', 'ADJ'),
 ('hour', 'NOUN'),
 ('guilt', 'NOUN'),
 ('utterance', 'NOUN'),
 ('accident', 'NOUN'),
 ('venom', 'NOUN'),
 ('adopt', 'VERB'),
 ('distorted', 'ADJ')]

In [38]:
# C2:SanityCheck

TDM.shape

(6262, 725)

__C3.__ _(8 pts)_ Next, your job is to define two functions. The first is `sim(u,v)`, which shoud take two arbitrary numeric vectors and compute/output the `cosine_similarity`, as described in __Section 1.1.2.10__.  

The second function is `term_sims(i, TDM)`, which should utilize the first function (`sim` function) to output a list of cosine similarity values (`sim_values`) between the word/row `i` and all others (rows) in the `TDM`.

Note: each of these functions can be straightforwardly completed using a single line of code! Exhibit your knowledge of comprehensions and vectorization!

In [39]:
# C3:Function(4/8)
import numpy as np
def sim(u,v):

    #---Your code starts here
    cosine_similarity = np.dot(u, v) / (np.linalg.norm(u) * np.linalg.norm(v))
    #---Your code ends here

    return cosine_similarity

In [40]:
# C3:SanityCheck

print("Exactly similar:", sim(np.array([1,2,3]), np.array([1,2,3])))
print("Exactly dissimilar:", sim(np.array([1,2,3]), np.array([-1,-2,-3])))
print("In the middle:", sim(np.array([1,1]), np.array([-1,1])))

Exactly similar: 1.0
Exactly dissimilar: -1.0
In the middle: 0.0


In [41]:
# C3:Function(4/8)

def term_sims(i, TDM):

    #---Your code starts here
    sim_values = [sim(TDM[i], TDM[j]) for j in range(TDM.shape[0])]
    #---Your code ends here

    return sim_values

In [42]:
# C3:SanityCheck

# Compare word/row 0 to all other (rows) in the TDM
term_sims(0, TDM)

[1.0,
 0.0,
 0.0,
 0.07802743146408705,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.05407380704358752,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.034518691538566515,
 0.0,
 0.0,
 0.0,
 0.10362129429155138,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.07647191129018724,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.08606629658238706,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.13608276348795434,
 0.0,
 0.0,
 0.0,
 0.0629940788348712,
 0.0,
 0.0,
 0.2357022603955158,
 0.0,
 0.07597371763975863,
 0.0,
 0.0,
 0.0,
 0.03456505649101418,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0563436169819011,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.3333333333333333,
 0.0,
 0.0,
 0.3333333333333333,
 0.0,


__C4.__ _(7 pts)_ Finally, your goal now is to a write function, `most_similar(term, terms, TDM, top = 25)`, that utilizes `term_sims` to output a sorted list of the `top = N` terms (`top_n_terms`) most similar to one specified (`term`). The output data type should be a list of lists, with each inner list representing information for a similar term as: `[row_ix, similarity, term]`.

\[Hint: to locate the row containing the term of interest, utilize the list `.index()` method in application to the `terms` argument.\]

In [43]:
# C4:Function(6/7)

def most_similar(term, terms, TDM, top = 25):

    #---Your code starts here---
    try:
        term_index = terms.index(term)
    except ValueError:
        raise ValueError(f"The term '{term}' is not in the terms list.")

    term_vector = TDM[term_index]

    similarities = []
    for idx, other_term_vector in enumerate(TDM):
        if idx != term_index:
            similarity = sim(term_vector, other_term_vector)
            similarities.append([idx, similarity, terms[idx]])

    similarities.sort(key=lambda x: x[1], reverse=True)
    top_n_terms = similarities[:top]
    #---Your code ends here---

    return top_n_terms

Now, let's test your functions utility on a `TDM` produced for `book_id = 84` and exhibit the top 25 similar terms to both of `('monster', 'NOUN')` and `('beautiful', 'ADJ')`.

In [44]:
# C4:SanityCheck

most_similar(('monster', 'NOUN'), terms, TDM, top = 25)

[[143, 0.34299717028501764, ('posterity', 'NOUN')],
 [1816, 0.34299717028501764, ('formation', 'NOUN')],
 [2087, 0.34299717028501764, ('mutilate', 'VERB')],
 [2447, 0.34299717028501764, ('correct', 'VERB')],
 [4126, 0.34299717028501764, ('asseveration', 'NOUN')],
 [2522, 0.337510803485021, ('I', 'PRON')],
 [726, 0.3102526139970115, ('hideous', 'ADJ')],
 [4820, 0.2970442628930023, ('demoniacal', 'ADJ')],
 [6036, 0.2970442628930023, ('forgetfulness', 'NOUN')],
 [5266, 0.29194407123319405, (' ', 'SPACE')],
 [1979, 0.28020184134909754, ('that', 'SCONJ')],
 [810, 0.2794815423200416, ('!', 'PUNCT')],
 [3608, 0.27370899867041026, ('\n', 'SPACE')],
 [4612, 0.2718318190198858, ('.', 'PUNCT')],
 [518, 0.27086782501514084, ('the', 'DET')],
 [4949, 0.27083418336873366, ('and', 'CCONJ')],
 [67, 0.2585438449975096, ('neck', 'NOUN')],
 [1500, 0.25724787771376323, ('narrative', 'NOUN')],
 [5530, 0.25724787771376323, ('loathsome', 'ADJ')],
 [4382, 0.25250143182442775, (',', 'PUNCT')],
 [2334, 0.2516271

In [45]:
# C4:SanityCheck

most_similar(('beautiful', 'ADJ'), terms, TDM, top = 25)

[[2743, 0.40824829046386296, ('beneath', 'ADV')],
 [338, 0.37499999999999994, ('castle', 'NOUN')],
 [719, 0.37499999999999994, ('singularly', 'ADV')],
 [3908, 0.37499999999999994, ('horrid', 'ADJ')],
 [4784, 0.37499999999999994, ('Rotterdam', 'PROPN')],
 [393, 0.35355339059327373, ('fifth', 'ADJ')],
 [525, 0.35355339059327373, ('luxuriance', 'NOUN')],
 [571, 0.35355339059327373, ('dun', 'PROPN')],
 [676, 0.35355339059327373, ('Mannheim', 'PROPN')],
 [750, 0.35355339059327373, ('Mainz', 'PROPN')],
 [858, 0.35355339059327373, ('whence', 'ADP')],
 [1162, 0.35355339059327373, ('meandering', 'ADJ')],
 [1605, 0.35355339059327373, ('straight', 'ADJ')],
 [2582, 0.35355339059327373, ('populous', 'ADJ')],
 [3000, 0.35355339059327373, ('shrivel', 'VERB')],
 [3754, 0.35355339059327373, ('shipping', 'NOUN')],
 [3789, 0.35355339059327373, ('variegate', 'VERB')],
 [3808, 0.35355339059327373, ('delineate', 'ADJ')],
 [3848, 0.35355339059327373, ('pearly', 'ADJ')],
 [3913, 0.35355339059327373, ('willowy

In [46]:
# C4:Inline

# Comment on the ordered results returned in the sanity checks.
# Do you think the algorithm is exhibiting sensible results? print "Yes" or "No"
print("Yes")

Yes
